# Refleksja

## Struktura pliku .class, dostęp do informacji o typie, polach i metodach
## Dynamiczne ładowanie klas

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@o2.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

```java
class DummyIterator implements Iterator {
    private int index = 0;
    
    @Override
    public boolean hasNext(){
        return false;
    }
    
    @Override
    public Object next(){
        return null;
    }
}
```

```
javac DummyIterator.java
```

```
cat DummyIterator.class
```

```
����4
	indexI<init>()VCodeLineNumberTablehasNext()Znext()Ljava/lang/Object;
SourceFileDummyIterator.java
                             	
DummyIteratorjava/lang/Objectjava/util/Iterator      
&
*�*��



�
       
�
%        
```

```
javap DummyIterator
```

```java
Compiled from "DummyIterator.java"
class DummyIterator implements java.util.Iterator {
  DummyIterator();
  public boolean hasNext();
  public java.lang.Object next();
}

```

<img src="img/cat4.jpg"/>

# Informacje o typie

```java
Class<?> klass = null;
klass = int.class;
klass = Integer.class;
klass = "Ala ma kota".getClass();
klass = new String[0].getClass();
klass = new DummyIterator().getClass();
```

# Klasa nadrzędna

```java
Class<?> klass = null;
klass = new DummyIterator().getClass();
klass = klass.getSuperclass();
```

# Wspólna klasa nadrzędna

```java
class A {}
class B1 extends A {}
class C1 extends B1 {}
class B2 extends A {}
class C2 extends B2 {}
```

```java
List<Class<?>> ancestors(Class<?> klass){
    List<Class<?>> result = new LinkedList<>();
    while(klass != Object.class){
        result.add(klass);
        klass = klass.getSuperclass();
    }
    result.add(Object.class);
    Collections.reverse(result);
    return result;
}
```

```java
ancestors(C2.class);
```

```java
Class<?> commonAncestor(Class<?> aClass, Class<?> bClass){
    List<Class<?>> aAncestors = ancestors(aClass);
    List<Class<?>> bAncestors = ancestors(bClass);
    for(int i = 0; i < aAncestors.size() && i < bAncestors.size(); i++){
        if(aAncestors.get(i) != bAncestors.get(i)){
            return aAncestors.get(i-1);
        }
    }
    return aAncestors.get(aAncestors.size()-1);
}
```

```java
commonAncestor(C1.class, C2.class);
```

<img src="img/cat1.jpg"/>

# Dynamiczne ładowanie klas

```java
interface IVehicle {
    void go();
}
```

```java
class Car implements IVehicle {
    public void go(){
        System.out.println("Brum, brum");
    }
}
```

```java
class Airplane implements IVehicle {
    public void go(){
        System.out.println("Bzzzzzz");
    }
}
```

```java
class Rocket implements IVehicle {
    public void go(){
        System.out.println("BUUUUUUUUUUUM");
    }
}
```

```java
class VehicleSystem {
    public static void main(String[] args){
        List<IVehicle> vehicles = new LinkedList<>();
        for(String vehicleName : args){
            try {
                Class<IVehicle> klass = (Class<IVehicle>) Class.forName(vehicleName);
                vehicles.add(klass.newInstance());
            } catch(ClassNotFoundException ex) {
                System.out.println("The class " + vehicleName + 
                    " has not been found. ");
                return;
            } catch(InstantiationException | IllegalAccessException ex) {
                System.out.println("The class " + vehicleName + 
                    " cannot be instantiated. ");
                return;
            }
            
        }
        for(IVehicle vehicle : vehicles) {
            vehicle.go();
        }
    }
}

```

```
java VehicleSystem Car Rocket Airplane Car
```

<img src="img/cat2.jpg"/>

# Dostęp do składowych

* newInstance()
* getMethod(...)
* getMethods()
* getConstructor(...)
* getConstructors()
* getField(...)
* getFields()

```java
void prettyPrint(Object[] elements){
    for(Object element : elements)
        System.out.println(element);
}
```

```java
Class<?> klass = DummyIterator.class;

prettyPrint(klass.getMethods());
prettyPrint(klass.getConstructors());
prettyPrint(klass.getFields());
```

# Dynamiczna instancjacja

```java
import java.lang.reflect.*;

Constructor constructor = klass.getConstructor(new Class<?>[0]);
Iterator dummyIterator = (Iterator) constructor.newInstance();
```

```java
class DummyIterator implements Iterator {
    private int index = 0;
    
    public DummyIterator(){
    }
    
    public DummyIterator(int index){
        this.index = index;
    }

    @Override
    public boolean hasNext(){
        return false;
    }

    @Override
    public Object next(){
        return null;
    }
}
```

```java
Class<?> klass = DummyIterator.class;

Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);
```

# Dynamiczne wywołanie metody

```java
Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
hasNextMethod.getName();
hasNextMethod.getParameterTypes();
```

```java
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
hasNextMethod.invoke(dummyIterator);
```

# Dynamiczny dostęp do pól

```java
Field indexField = klass.getField("index");
indexField.getName();
indexField.getModifiers();
```

```
Field indexField = klass.getField("index");
|  java.lang.NoSuchFieldException thrown: index
|        at Class.getField (Class.java:1891)
|        at (#300:1)
```

```java
Field indexField = klass.getDeclaredField("index");
indexField.getName();
indexField.getModifiers();
```

```java
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Field indexField = klass.getDeclaredField("index");
indexField.get(dummyIterator);
```

```
indexField.get(dummyIterator);
|  java.lang.IllegalAccessException thrown: class REPL.$JShell$151 cannot access a member of class REPL.$JShell$19D$DummyIterator with modifiers "private"
|        at Reflection.throwIllegalAccessException (Reflection.java:405)
|        at Reflection.throwIllegalAccessException (Reflection.java:396)
|        at Reflection.ensureMemberAccess (Reflection.java:98)
|        at AccessibleObject.slowCheckMemberAccess (AccessibleObject.java:355)
|        at AccessibleObject.checkAccess (AccessibleObject.java:347)
|        at Field.get (Field.java:406)
|        at (#313:1)
```

```java
indexField.setAccessible(true);
indexField.get(dummyIterator);
indexField.set(dummyIterator,2);
```

<img src="img/mouth.jpg"/>

# Dostęp do wszystkich składowych prywatnych

* getDeclaredMethod(...)
* getDeclaredMethods()
* getDeclaredConstructor(...)
* getDeclaredConstructors()
* getDeclaredField(...)
* getDeclaredFields()

Uwaga: składowe dziedziczone nie są zwracane

# Dynamiczne proxy

```java
import java.lang.reflect.*;

class MethodReplayer implements InvocationHandler {
    private List<Object> instances = new LinkedList<>();
    private List<Method> invokedMethods = new LinkedList<>();
    private List<Object[]> invokedArguments = new LinkedList<>();
    
    public Object invoke(Object proxy, Method method, Object[] args) throws Throwable{
        if(!method.getName().equals("toString")){
            instances.add(proxy);
            invokedMethods.add(method);
            invokedArguments.add(args);
        }
        return method.invoke(proxy, args);
    }
    
    public void replay() throws Throwable {
        for(int i = 0; i < invokedMethods.size(); i++){
            invokedMethods.get(i).invoke(instances.get(i), invokedArguments.get(i));
        }
    }
}
```

```java
MethodReplayer replayer = new MethodReplayer();
List<String> list = (List<String>) java.lang.reflect.Proxy.newProxyInstance(List.class.getClassLoader(),
        new Class[] { List.class },
        replayer);

list.add("dog");
list.add("cat");
System.out.println(list);

replayer.replay();
System.out.println(list);
```

<img src="img/cat3.jpg"/>

# Uchwyty do metod

* w API refleksji sygnatura argumentów metody jest wskazywana przez `Class[]`
* w nowym API mamy nowy typ `MethodType`:
  * brak nazwy metody
  * brak typu adresata
  * tylko typy argumentów oraz typ zwracanej wartości

```java
import java.lang.invoke.*;
import java.lang.invoke.MethodHandles.Lookup;

// np. String toString()
MethodType toStringType = MethodType.methodType(String.class);

// np. Integer parseInt(String str)
MethodType fromStringToInt = MethodType.methodType(Integer.class, String.class);
```

```java
class CallMeMaybe {
    public String toString(){
        return "call me maybe";
    }
    
    public Lookup getLookup(){
        return MethodHandles.lookup();
    }
}
```

```java
CallMeMaybe maybe = new CallMeMaybe();
Lookup maybeLookup = maybe.getLookup();

MethodHandle maybeHandle = maybeLookup.findVirtual(CallMeMaybe.class, "toString", toStringType);
maybeHandle.invoke(maybe);
```

![Pytania? ](http://cliparts.co/cliparts/qcB/jqg/qcBjqgxc5.jpg)